In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from collect_functions import *
from constants import *

%load_ext autoreload
%autoreload 2

## Process raw data and organize into dataframes

In [ ]:
# get data files
filenames = [y for x in os.walk(RAW_DATA_PATH) for y in glob(os.path.join(x[0], '*.json'))]

# write dataframes to disk
store_all_df(filenames, agg_features="max", agg_survey=False)
store_all_df(filenames, agg_features="slope", agg_survey=False)
store_all_time_series(filenames)

## Read dataframes

In [ ]:
# read main dataframes
df_features_max = pd.read_csv(DATAFRAMES_PATH + "df_features_agg_max.csv")
df_features_slope = pd.read_csv(DATAFRAMES_PATH + "df_features_agg_slope.csv")

df_generals = pd.read_csv(DATAFRAMES_PATH + "df_generals.csv")
df_answers = pd.read_csv(DATAFRAMES_PATH + "df_answers.csv")

In [ ]:
video_names = {1: "arm", 2: "pepper", 3: "sophia"}
def replace_video_id(df):
    df["video"] = df.video_id.replace(video_names)
    df.drop("video_id", axis=1, inplace=True)
    return df

df_features_max = replace_video_id(df_features_max)
df_features_slope = replace_video_id(df_features_slope)
df_answers = replace_video_id(df_answers)

# set indexes of dataframes
df_features_max.set_index(["user_id", "video"], inplace=True)
df_features_slope.set_index(["user_id", "video"], inplace=True)
df_answers.set_index(["user_id", "video"], inplace=True)
df_generals.set_index("user_id", inplace=True)

## Data cleaning

In [ ]:
# minimum length of the time series for proper analysis
threshold = 100
# non-meaningful data
indices_to_drop = df_features_max[df_features_max.nb_timestamps < 100].index

# remove such videos, and their corresponding answers in the survey
df_features_max.drop(indices_to_drop, inplace=True)
df_features_slope.drop(indices_to_drop, inplace=True)
df_answers.drop(indices_to_drop, inplace=True)

df_features_max.drop("nb_timestamps", axis=1, inplace=True)
df_features_slope.drop("nb_timestamps", axis=1, inplace=True)

assert(len(df_answers) == len(df_features_max))

## Explore data

In [ ]:
NB_PARTICIPANTS = len(df_generals)
print("We had {0} participants".format(NB_PARTICIPANTS))

In [ ]:
plt.hist(df_generals.age, bins=range(10, 90, 5))
plt.title("Age distribution")
plt.ylabel("Frequency")
plt.xlabel("Age")
plt.savefig("plot/age.png")

In [ ]:
df_generals.robotRealLife = df_generals.robotRealLife.replace({ 0: "No", 1: "Yes" })
general = df_generals[["gender", "robotRealLife"]]

general_frequency = pd.crosstab(
    [np.repeat("gender", NB_PARTICIPANTS), general.gender],
    [np.repeat("robotRealLife", NB_PARTICIPANTS), general.robotRealLife],
    margins=True
)

general_frequency.index.names = [None, None]
general_frequency.columns.names = [None, None]

general_frequency

In [ ]:
df_answers.head()

In [ ]:
df_answers.describe()

In [ ]:
df_features_slope.head()

In [ ]:
df_features_slope.describe()

## Pearson correlation analysis

In [ ]:
# Aggregate overall survey scale
overall_score = df_answers.mean(axis=1)

# Aggregate answers for each subscale
anthropomorphism_score = df_answers[ANTHROPOMORPHISM_COLUMNS].mean(axis=1)
animacy_score = df_answers[ANIMACY_COLUMNS].mean(axis=1)
likeability_score = df_answers[LIKEABILITY_COLUMNS].mean(axis=1)
intelligence_score = df_answers[INTELLIGENCE_COLUMNS].mean(axis=1)

In [ ]:
df_features_max.head()

In [ ]:
corr_max = pd.DataFrame({
    "anthropomorphism": df_features_max.corrwith(anthropomorphism_score),
    "animacy": df_features_max.corrwith(animacy_score),
    "likeability": df_features_max.corrwith(likeability_score),
    "intelligence": df_features_max.corrwith(intelligence_score),
    "overall": df_features_max.corrwith(overall_score)
})

In [ ]:
print("max correlation", corr_max.max().max())
print("min correlation", corr_max.min().min())

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Correlations of max-aggregated features with the survey response", fontsize=16, pad=25)
sns.heatmap(corr_max.round(2).T, vmin=-1, vmax=1, cmap="coolwarm", square=True, annot=True)
plt.savefig("plot/corr_features_max_survey.png");

In [ ]:
corr_slope = pd.DataFrame({
    "anthropomorphism": df_features_slope.corrwith(anthropomorphism_score),
    "animacy": df_features_slope.corrwith(animacy_score),
    "likeability": df_features_slope.corrwith(likeability_score),
    "intelligence": df_features_slope.corrwith(intelligence_score),
    "overall": df_features_slope.corrwith(overall_score)
})

In [ ]:
print("max correlation", corr_slope.max().max())
print("min correlation", corr_slope.min().min())

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Correlations of slope-aggregated features with the survey response", fontsize=16, pad=25)
sns.heatmap(corr_slope.round(2).T, vmin=-1, vmax=1, cmap="coolwarm", square=True, annot=True)
plt.savefig("plot/corr_features_slope_survey.png");

In [ ]:
corr_features_max = df_features_max.corr()
corr_features_slope = df_features_slope.corr()

In [ ]:
print("max correlation", corr_features_max.replace(1, 0).max().max())
print("min correlation", corr_features_max.replace(1, 0).min().min())

In [ ]:
plt.figure(figsize=(13, 11))
plt.title("Pairwise correlations of max-aggregated features", fontsize=20, pad=25)
sns.heatmap(corr_features_max.round(2).T, vmin=-1, vmax=1, cmap="coolwarm", square=True, annot=True)
plt.savefig("plot/corr_pairwise_features_max.png");

In [ ]:
plt.figure(figsize=(13, 11))
plt.title("Pairwise correlations of slope-aggregated features", fontsize=20, pad=25)
sns.heatmap(corr_features_slope.round(2).T, vmin=-1, vmax=1, cmap="coolwarm", square=True, annot=True)
plt.savefig("plot/corr_pairwise_features_slope.png");

In [ ]:
df_features_max.columns.name = "feature"
df_features_slope.columns.name = "feature"

df_features_max_transformed = (
    df_features_max
    .drop("eyeClosure", axis=1) # drop eyeClosure to make plot readable
    .stack()
    .to_frame("value")
    .reset_index()
)

df_features_slope_transformed = (
    df_features_slope
    .drop("eyeClosure", axis=1) # drop eyeClosure to make plot readable
    .stack()
    .to_frame("value")
    .reset_index()
)

In [ ]:
df_features_slope_transformed.head()

In [ ]:
# seed for reproducibility
seed = 42

plt.figure(figsize=(11, 4))
sns.barplot(x="feature", hue="video", y="value", data=df_features_max_transformed, seed=seed, errwidth=0.5)
plt.title("Mean with 95% CI of max-aggregated features per video clip", fontsize=16, pad=10)
plt.xticks(rotation=90)
plt.xlabel(None)
plt.legend(loc="upper right")
plt.savefig("plot/mean_max_features_per_video.png", bbox_inches="tight");

In [ ]:
plt.figure(figsize=(11, 4))
sns.barplot(x="feature", hue="video", y="value", data=df_features_slope_transformed, seed=seed, errwidth=0.5)
plt.title("Mean with 95% CI of slope-aggregated features per video clip", fontsize=16, pad=25)
plt.xticks(rotation=90)
plt.xlabel(None)
plt.savefig("plot/mean_slope_features_per_video.png", bbox_inches="tight");

In [ ]:
scores = pd.concat(
    objs={
        "anthropomorphism": anthropomorphism_score,
        "animacy": animacy_score,
        "likeability": likeability_score,
        "intelligence": intelligence_score,
        "overall": overall_score
    },
    axis=1
)

In [ ]:
scores.columns.name = "godspeed_scale"

scores_transformed = (
    scores
    .stack()
    .to_frame("value")
    .reset_index()
)


scores_transformed.head()

In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(x="godspeed_scale", hue="video", y="value", data=scores_transformed, seed=seed, errwidth=1)
plt.title("Mean with 95% CI of overall and subscale survey responses per video clip", fontsize=16, pad=25)
plt.xlabel(None)
plt.savefig("plot/mean_survey_per_video.png", bbox_inches="tight");

## Model training

In [ ]:
from sklearn.linear_model import LassoCV

def lasso_cv(X, y, test_size=0.2, seed=42):
    """
    Splits the data into training and test samples.
    Fits the training samples to a Lasso linear model and
    selects the best model by cross-validation.
    Returns the R-squared coefficient of determination of
    the test prediction.
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
    model = LassoCV(random_state=seed).fit(X_train, y_train)
    r2 = model.score(X_test, y_test)
    print(len([x for x in model.coef_ if x != 0]), " nonzero coefficients out of 14")
    return r2

In [ ]:
results_max = {}
results_slope = {}
results_mean = {}

for survey_scale in scores.columns:
    target = scores[survey_scale]
    results_max[survey_scale] = lasso_cv(df_features_max, target)
    results_slope[survey_scale] = lasso_cv(df_features_slope, target)

In [ ]:
pd.DataFrame.from_dict(results_slope, orient='index', columns=["R-squared"])

In [ ]:
pd.DataFrame.from_dict(results_max, orient='index', columns=["R-squared"])

In [ ]:
pd.DataFrame.from_dict(results_slope, orient='index', columns=["R-squared"])

## Try to predict video_id from features

In [ ]:
X = df_features_max[CONSIDERED_FEATURES]
y = df_features_max.index.get_level_values(1)
n = 100
err = []
# repeat with different seed
for i in range(n):
    # split data into training and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=i)

    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    err.append((y_test == y_pred).sum())
print("Average accuracy: {0}%".format(np.mean(err)/len(y_test) *100))

In [ ]:
X = df_features_slope[CONSIDERED_FEATURES]
y = df_features_max.index.get_level_values(1)
n = 100
err = []
# repeat with different seed
for i in range(n):
    # split data into training and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=i)

    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    err.append((y_test == y_pred).sum())
print("Average accuracy: {0}%".format(np.mean(err)/len(y_test) *100))

### Try to predict video_id from answers

In [ ]:
X = df_answers
y = df_features_max.index.get_level_values(1)
n = 100
err = []
# repeat with different seed
for i in range(n):
    # split data into training and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=i)

    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    err.append((y_test == y_pred).sum())
print("Average accuracy: {0}%".format(np.mean(err)/len(y_test) *100))

### Plot feature

In [ ]:
fig = plt.figure(figsize=(20,10))

feature_name = "smile"
t = pd.read_csv(DATAFRAMES_PATH + "df_" + feature_name + "_3.csv")
t.drop("video_id",inplace=True, axis=1)
t.set_index("user_id", inplace=True)
t[:15].T.plot()
plt.legend(title="user_id", loc='best', ncol=3, bbox_to_anchor=(1, 1, 0, 0))
plt.ylabel("facial action score")
plt.title("Smiling of 15 participants while watching Sophia")
plt.savefig("plot/smile.png", bbox_inches="tight");

### Features for each video and user

In [ ]:
X = df_features_max.values[:,1:]
pca = PCA(n_components=2)
y = pca.fit_transform(X)
vid1 = plt.scatter(y[::3][:,0], y[::3][:,1])
vid2 = plt.scatter(y[1::3][:,0], y[1::3][:,1])
vid3 = plt.scatter(y[2::3][:,0], y[2::3][:,1])

plt.legend((vid1,vid2,vid3), video_names.values(), fontsize=8)

plt.show()

In [ ]:
X = df_features_slope.values[:,1:]
pca = PCA(n_components=2)
y = pca.fit_transform(X)
vid1 = plt.scatter(y[::3][:,0], y[::3][:,1])
vid2 = plt.scatter(y[1::3][:,0], y[1::3][:,1])
vid3 = plt.scatter(y[2::3][:,0], y[2::3][:,1])

plt.legend((vid1,vid2,vid3), video_names.values(), fontsize=8)

plt.show()

### Answers for each video and user

In [ ]:
X = df_answers.values
pca = PCA(n_components=2)
y = pca.fit_transform(X)
vid1 = plt.scatter(y[::3][:,0], y[::3][:,1])
vid2 = plt.scatter(y[1::3][:,0], y[1::3][:,1])
vid3 = plt.scatter(y[2::3][:,0], y[2::3][:,1])

plt.legend((vid1,vid2,vid3), video_names.values(), fontsize=8)
plt.title("PCA plot of overall survey response")

plt.show()

In [ ]:
PCA().fit(X).explained_variance_ratio_